In [3]:
import requests
import pandas as pd
import numpy as np

#maximum items to query at once is 1000
#implement a skip variable to loop over all transactions returning 1000 transactions for each iteration. -> While loop
def get_messarisubg_uniswap_mints(skip =0 ,limit=1000):

    url = "https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum"

    query = """{
  deposits(skip: """ +str(skip)+""",first: """ +str(limit)+""",orderBy: timestamp, orderDirection: desc) {
    amountUSD
    blockNumber
    from
    to
    hash
    inputTokenAmounts
    inputTokens {
      decimals
      lastPriceUSD
      name
      symbol
    }
    timestamp
    pool {
      inputTokenBalances
      name
      symbol
      createdTimestamp
      id
    }
  }
}"""
    headers = {"Content-Type": "application/json"} #define in the request https header the content type: we want to send a json query in the body (there are a total of 4 types of http header types)
    #HTTP POST is a request method of an Hypertext Transfer Protocol (HTTP) and is used to create or update a resource on the server. The HTTP POST method is used to submit web forms and files and images to the server.
    response = requests.post(url, headers=headers, json={"query": query}) #define in the http request body the json data we want to send to the server

    if response.status_code != 200:
        raise Exception("Failed to retrieve UniSwap exchanges: " + response.text)

    return response.json()

#get more than first 1000 rows
skip_counter = 0
list_of_df = []
while True:
    try: 

        result = get_messarisubg_uniswap_mints(skip = skip_counter)
        list_of_df.append(pd.json_normalize(result["data"]["deposits"]))
        skip_counter += 1000
        
        
    except Exception: 
      break



df_mints = pd.concat(list_of_df).reset_index(drop = True)

df_mints.head() #gives me 6000 entries for the last 4 hours. MAx to query from thegraph at once is 6000 mints. 

,amountUSD,blockNumber,from,to,hash,inputTokenAmounts,inputTokens,timestamp,pool.inputTokenBalances,pool.name,pool.symbol,pool.createdTimestamp,pool.id
0,307.166616242267181508805092186728,16613399,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x69cd4a4d56e20385de9cde7e9f9a262d274855657e20...,"[700028, 100557986415394389]","[{'decimals': 8, 'lastPriceUSD': '21870.402770...",1676213567,"[329383670739, 78973306115142044059150]",Uniswap V3 Wrapped BTC/Wrapped Ether 0.3%,Wrapped BTC/Wrapped Ether,1620158974,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
1,2743.384072058829984425868851207256,16613396,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99132b53ab44694eeb372e87bced3929e4ab8456,0x2fb5a1eb2607925e839e74d1b7bc78c78a3ffda1c483...,"[763089627290, 1790571458838514526]","[{'decimals': 8, 'lastPriceUSD': '0', 'name': ...",1676213531,"[161179764646067, 256681323115485117996]",Uniswap V3 SingularityNET Token/Wrapped Ether 1%,SingularityNET Token/Wrapped Ether,1622422743,0x99132b53ab44694eeb372e87bced3929e4ab8456
2,576.4887743027687299096402625701283,16613392,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xa32650483e4b5d51416c41a3870b0e4a53c7e09e,0xf431b5bbe6990a726edc8574925fa445263d65fd7bc0...,"[376266799869792340, 30009292596767858415301]","[{'decimals': 18, 'lastPriceUSD': '1532.127667...",1676213483,"[26128653890458579556, 253879556591612626705657]",Uniswap V3 Wrapped Ether/GOVI 0.3%,Wrapped Ether/GOVI,1620438587,0xa32650483e4b5d51416c41a3870b0e4a53c7e09e
3,2451.404267407120642130849350146653,16613390,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x63818bbdd21e69be108a23ac1e84cbf66399bd7d,0xf4c3133871b1b5ac22f08e9009e38d4e939aac78c9b9...,"[392575132485088174, 1600000000000000000]","[{'decimals': 18, 'lastPriceUSD': '0', 'name':...",1676213459,"[28593169660797273363, 229391474036539334199]",Uniswap V3 Liquid staked Ether 2.0/Wrapped Eth...,Liquid staked Ether 2.0/Wrapped Ether,1654851812,0x63818bbdd21e69be108a23ac1e84cbf66399bd7d
4,0.001991878746336328771705419470851625,16613389,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0x04eb04963c5d9581398a48442c007396cea0bab3db96...,"[1000, 647386485876]","[{'decimals': 6, 'lastPriceUSD': '1', 'name': ...",1676213447,"[77446678071577, 60156916490669841319418]",Uniswap V3 USD Coin/Wrapped Ether 0.3%,USD Coin/Wrapped Ether,1620169800,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8


In [4]:
df_mints.hash[0]

'0x69cd4a4d56e20385de9cde7e9f9a262d274855657e20b5971ecd2c701b4e8bd0'